<a href="https://colab.research.google.com/github/aneesha17/dino/blob/main/dino_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


Creating checkpoints

In [2]:
import os
# from google.colab import drive
# drive.mount('/content/drive')
checkpoints = '/content/imagenet/'
if not os.path.exists(checkpoints):
    os.makedirs(checkpoints)

Clone the repository

In [3]:
!git clone https://github.com/aneesha17/dino/

Cloning into 'dino'...
remote: Enumerating objects: 199, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 199 (delta 71), reused 64 (delta 59), pack-reused 107
Receiving objects: 100% (199/199), 24.47 MiB | 14.09 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [5]:
import gdown
!gdown --id 18jJPqjo1OKS_FQQngj4Arnqf8zAmosbc


/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=18jJPqjo1OKS_FQQngj4Arnqf8zAmosbc
To: /content/imagenette2-160.zip
100% 107M/107M [00:02<00:00, 40.9MB/s]


In [6]:
!unzip /content/imagenette2-160.zip

Streaming output truncated to the last 5000 lines.
  inflating: imagenette2-160/train/n03445777/n03445777_8446.JPEG  
  inflating: imagenette2-160/train/n03445777/n03445777_8448.JPEG  
  inflating: imagenette2-160/train/n03445777/n03445777_8474.JPEG  
  inflating: imagenette2-160/train/n03445777/n03445777_8477.JPEG  
  inflating: imagenette2-160/train/n03445777/n03445777_8484.JPEG  
  inflating: imagenette2-160/train/n03445777/n03445777_8544.JPEG  
  inflating: imagenette2-160/train/n03445777/n03445777_8546.JPEG  
  inflating: imagenette2-160/train/n03445777/n03445777_8599.JPEG  
  inflating: imagenette2-160/train/n03445777/n03445777_8616.JPEG  
  inflating: imagenette2-160/train/n03445777/n03445777_8624.JPEG  
  inflating: imagenette2-160/train/n03445777/n03445777_8628.JPEG  
  inflating: imagenette2-160/train/n03445777/n03445777_866.JPEG  
  inflating: imagenette2-160/train/n03445777/n03445777_8663.JPEG  
  inflating: imagenette2-160/train/n03445777/n03445777_8677.JPEG  
  inflating:

In [7]:
%cd dino

/content/dino


Install timm library

In [8]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.6 MB/s eta 0:00:00


Loading models

In [9]:
import torch
vits16 = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
vits8 = torch.hub.load('facebookresearch/dino:main', 'dino_vits8')
vitb16 = torch.hub.load('facebookresearch/dino:main', 'dino_vitb16')
vitb8 = torch.hub.load('facebookresearch/dino:main', 'dino_vitb8')
xcit_small_12_p16 = torch.hub.load('facebookresearch/dino:main', 'dino_xcit_small_12_p16')
xcit_small_12_p8 = torch.hub.load('facebookresearch/dino:main', 'dino_xcit_small_12_p8')
xcit_medium_24_p16 = torch.hub.load('facebookresearch/dino:main', 'dino_xcit_medium_24_p16')
xcit_medium_24_p8 = torch.hub.load('facebookresearch/dino:main', 'dino_xcit_medium_24_p8')
resnet50 = torch.hub.load('facebookresearch/dino:main', 'dino_resnet50')

Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_deitsmall16_pretrain.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 166MB/s]
Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_deitsmall8_pretrain/dino_deitsmall8_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_deitsmall8_pretrain.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 192MB/s]
Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_vitbase16_pretrain/dino_vitbase16_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_vitbase16_pretrain.pth
100%|██████████| 327M/327M [00:02<00:00, 170MB/s]
Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main
Downloading: 

In [ ]:
!python -m torch.distributed.run --nproc_per_node=1 main_dino.py --arch vit_small --data_path /content/imagenette2-160 --output_dir /content/data/output

Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main
| distributed init (rank 0): env://
git:
  sha: cb88175ea889432d4c803e07ebb765bae98b844a, status: clean, branch: main

arch: vit_small
batch_size_per_gpu: 64
clip_grad: 3.0
data_path: /content/imagenette2-160
dist_url: env://
drop_path_rate: 0.1
epochs: 100
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 0
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
norm_last_layer: True
num_workers: 10
optimizer: adamw
out_dim: 65536
output_dir: /content/data/output
patch_size: 16
rank: 0
saveckp_freq: 20
seed: 0
teacher_temp: 0.04
use_bn_in_head: False
use_fp16: True
warmup_epochs: 10
warmup_teacher_temp: 0.04
warmup_teacher_temp_epochs: 0
weight_decay: 0.04
weight_decay_end: 0.4
world_size: 1
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested

In [ ]:
!pip install submitit

Evaluation on knn


In [ ]:
!python -m torch.distributed.run --nproc_per_node=1 eval_knn.py --data_path /content/imagenette2-160

Evaluation on linear

In [ ]:
!python -m torch.distributed.run --nproc_per_node=1 eval_linear.py --data_path /content/imagenette2-160